<a href="https://colab.research.google.com/github/bannis14/Portfolio-Projects/blob/main/Time_Series_Forecast/RNN_LSTM_Model_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries and Data
  *     Need to Connect to GPU



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [ ]:
!pip install pyyaml
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.8/784.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import ParameterGrid
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from sklearn.metrics import mean_squared_error

In [ ]:
if gpu_device_name:
    print(f"Connected to GPU: {gpu_device_name}")
else:
    print("No GPU available. Switch to a GPU runtime in Colab.")

In [ ]:
# Load the Data
df = pd.read_csv('/content/drive/MyDrive/Time Series Forecasting Product/nyc_data.csv', index_col=0, parse_dates=True)
future_df = pd.read_csv('/content/drive/MyDrive/Time Series Forecasting Product/future.csv', index_col=0, parse_dates=True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [ ]:
# Extract Regressors
X_train = df.iloc[:, 1:]
X_future = future_df.iloc[:, 1:]

In [ ]:
X = pd.concat([X_train, X_future])

In [ ]:
# Rename Variables
df = df.rename(columns={'Demand': 'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


## Prep for LTSM


In [ ]:
# Time Series Object
series = TimeSeries.from_series(df.y)
covariates = TimeSeries.from_dataframe(X)

In [ ]:
# year
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(),
                  freq=series.freq_str,
                  periods=X.shape[0]),
    attribute="year",
    one_hot=False)

In [ ]:
# Month
month_series = datetime_attribute_timeseries(year_series,
                                             attribute="month",
                                             one_hot=True)

In [ ]:
# Weekday
weekday_series = datetime_attribute_timeseries(year_series,
                                               attribute="weekday",
                                               one_hot=True)

In [ ]:
# Prepping the Scaler
transformer1 = Scaler()
transformer2 = Scaler()

In [ ]:
y_transformed = transformer1.fit_transform(series)

In [ ]:
covariates = covariates.stack(year_series)
covariates_transformed = transformer2.fit_transform(covariates)
covariates_transformed = covariates_transformed.stack(month_series)
covariates_transformed = covariates_transformed.stack(weekday_series)

## LSTM Model


In [ ]:
# get the best parameters
parameters = pd.read_csv("Forecasting Product/best_params_lstm.csv",
                         index_col=0)
parameters

,2
dropout,0.200000
hidden_dim,20.000000
input_chunk_length,15.000000
lr,0.001000
n_epochs,20.000000
n_rnn_layers,4.000000
training_length,20.000000
rmse,88.791181


In [ ]:
# Getting the parameters

n_rnn_layers = int(parameters.loc["n_rnn_layers"][0])
dropout = float(parameters.loc["dropout"][0])
hidden_dim = int(parameters.loc["hidden_dim"][0])
input_chunk_length = int(parameters.loc["input_chunk_length"][0])
lr = float(parameters.loc["lr"][0])
n_epochs = int(parameters.loc["n_epochs"][0])
training_length = int(parameters.loc["training_length"][0])

In [ ]:
# Model
model = RNNModel("LSTM",
                 hidden_dim=hidden_dim,
                 n_rnn_layers=n_rnn_layers,
                 dropout=dropout,
                 n_epochs=n_epochs,
                 optimizer_kwargs={'lr': lr},
                 random_state=1502,
                 training_length=training_length,
                 input_chunk_length=input_chunk_length,
                 pl_trainer_kwargs={"accelerator": "gpu", "devices": [0]},
                 save_checkpoints= True,
                 )

In [ ]:
# Fit the model to the data
model.fit(y_transformed,
          future_covariates=covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 3.5 K 
4 | V             | Linear           | 21    
---------------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


RNNModel(input_chunk_length=15, model=RNN, hidden_dim=20, n_rnn_layers=4, dropout=0.2, training_length=20, n_epochs=20, optimizer_kwargs={'lr': 0.001}, random_state=1502, pl_trainer_kwargs={'accelerator': 'gpu', 'devices': -1}, save_checkpoints=False)

## Predictions


In [ ]:
predictions_lstm = model.predict(n=len(future_df),
                                 future_covariates=covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
predictions_lstm = TimeSeries.pd_series(
    transformer1.inverse_transform(predictions_lstm)).rename("lstm")

In [ ]:
predictions_lstm

Date
2021-01-01     757.678595
2021-01-02     878.076908
2021-01-03     790.532398
2021-01-04     932.342795
2021-01-05     787.550850
2021-01-06     971.505666
2021-01-07     848.732532
2021-01-08     778.026554
2021-01-09     898.518911
2021-01-10     811.511686
2021-01-11     956.125926
2021-01-12     818.449575
2021-01-13     986.208565
2021-01-14     884.760508
2021-01-15     784.754258
2021-01-16     931.463367
2021-01-17     826.795464
2021-01-18     986.908833
2021-01-19     830.125437
2021-01-20    1015.937013
2021-01-21     898.169702
2021-01-22     801.358715
2021-01-23     944.638797
2021-01-24     848.605817
2021-01-25     998.904694
2021-01-26     857.156828
2021-01-27    1029.210062
2021-01-28     931.072171
2021-01-29     833.448175
2021-01-30     982.697302
2021-01-31     878.721729
Freq: D, Name: lstm, dtype: float64

In [ ]:
predictions_lstm.to_csv("Forecasting Product/Ensemble/predictions_lstm.csv")